<a href="https://colab.research.google.com/github/Elwing-Chou/tibame_online_0616/blob/main/tibame_night_20250708.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import urllib.request as req
import bs4 as bs

url = "https://www.ptt.cc/bbs/Beauty/M.1751962851.A.45A.html"
resp = req.urlopen(url)
content = resp.read()
html = bs.BeautifulSoup(content)
print(html)

HTTPError: HTTP Error 403: Forbidden